# Experiment 2-Stage

### Load Model & Dataset

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.models.simple_3d_model import LinearModel
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.eval.accuracies import keypoint_depth_pck, get_max_preds

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    reduced_limb_ranges = [[0,4],[4,7],[7,10],[10,13],[13,16],[16,19],[19,22],[22,25],[25,28]]
    if len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line
    elif len(points) == 28:
        points = np.array(points).T
        for num in range(len(reduced_limb_ranges)):
            visible_limb = []
            for x in range(reduced_limb_ranges[num][0],reduced_limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [3]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [4]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,_ ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 128

test_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Load Models
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model_2d = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model_2d = PoseResNet(block_class, layers, 62)

model_2d = model_2d.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model_2d.load_state_dict(torch.load(path))
model_2d.eval()


Using cuda device


PoseResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True

In [6]:
if reduceKeypoints:
    model_lift = LinearModel(2*28,28,1024) # Linear size 512 looked promising 256 too small
else:
    model_lift = LinearModel(2*62,62,512)

model_lift = model_lift.to(device)
path = "../models/lifting_depth/lifting_depth_model_20220323-150326_128"
model_lift.load_state_dict(torch.load(path))
model_lift.eval()

LinearModel(
  (w1): Linear(in_features=56, out_features=1024, bias=True)
  (batch_norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_stages): ModuleList(
    (0): Linear(
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=1024, out_features=1024, bias=True)
      (batch_norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (w2): Linear(in_features=1024, out_features=1024, bias=True)
      (batch_norm2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Linear(
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (w1): Linear(in_features=1024, out_features=1024, bias=True)
      (batch_norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (w2): Linear(in_features=1024, out_features=1024, bias=True)
      (batch_norm2)

In [ ]:
def test_func(model_2d,model_lift, ds, frac_noise=0.0,thr=5.0):
    test_epoch_acc, test_epoch_loss = 0, 0
    test_epoch_acc_kps = [0]*28
    batches = 0
    # Code get the first batch of results
    with torch.no_grad():
        for data in tqdm(ds, desc="2-Stage Experiment"):
            input = data['image'].to(device, dtype=torch.float)
            kp = data['key_points_2D'].to(device, dtype=torch.float)
            y = data['key_points_3D'][:,:,2].to(device)
            mask = data['visibility'].to(device)

            
            pred_heatmap =  model_2d(input)
            kps2d,_ = get_max_preds(pred_heatmap.detach().cpu().numpy())

            # Add noise to the 2d input add guassian noise to determine how sensitve the model is that will give us a reason to why the model is working so badly
            
            ## Prints the using GT Noisey
            # kps2d = kp.detach().cpu().numpy()
            
            kps2d = kps2d.reshape((len(kps2d), 56))

            if frac_noise > 0:
                for bat in range(len(kps2d)):
                    for col in range(len(kps2d[bat])):
                        mean = abs(kps2d[:,col].mean())
                        # print(mean)
                        noise = np.random.normal(0,frac_noise*mean,len(kps2d[:,col]))
                        kps2d[:,col] += noise

            pred_lift = model_lift(torch.from_numpy(kps2d).to(device))
            
            test_loss = torch.mean(((pred_lift - y)*mask)**2) # 
                    
            test_acc,test_acc_kps = keypoint_depth_pck(pred_lift.detach().cpu().numpy(),
                                        y.detach().cpu().numpy(), 
                                        mask.detach().cpu().numpy(),
                                        sungaya_dataset.std_3d[:,2], sungaya_dataset.means_3d[:,2], threshold = thr)
            test_epoch_loss += test_loss.item()
            test_epoch_acc += test_acc
            test_epoch_acc_kps += test_acc_kps
            batches +=1
            
    test_acc = test_epoch_acc/batches
    test_acc_kps = test_epoch_acc_kps/batches
    test_loss = test_epoch_loss/batches
    return test_acc, test_loss, test_acc_kps


In [ ]:
test_acc, test_loss, test_acc_kps = test_func(model_2d, model_lift, test_dataloader)
print(test_acc)
print(test_loss)


In [ ]:
# Plots the accuracies per KP at normal threshold
 
fig, ax = plt.subplots(figsize=(11,8))

ax.barh(np.array(label_names)[sungaya_dataset.reduced_kp], test_acc_kps, linewidth =5, edgecolor="white")
ax.invert_yaxis()
# ax.set(xlim=(-1, 8), xticks=np.arange(-1, 8),
#        ylim=(0, 1), yticks=np.arange(0, 0.5))
plt.title("2 Stage PCK Accuracies for every Keypoint on Synthetic Data")

plt.xlabel("Accuracies")
plt.ylabel("Keypoints")
plt.show()

In [ ]:
# Need to do an experiment where we vary the threshold value on the test set
thresholds = [0.01 ,0.1, 1.0, 2.5, 5.0, 10.0, 15.0, 20.0]
accs = []
losses = []
acc_kps = []
for thr in thresholds:
    test_acc, test_loss, test_acc_kps = test_func(model_2d, model_lift, test_dataloader, thr=thr)
    accs.append(test_acc)
    losses.append(test_loss)
    acc_kps.append(test_acc_kps)

In [ ]:
# plots PCK for different thresholds
fig, ax = plt.subplots()

ax.bar(list(map(str,thresholds)), accs, width=1, edgecolor="white", linewidth=5)

plt.title("2 Stage PCK Accuracies at Varying Thresholds on Synthetic Data")
plt.xlabel("Thresholds")
plt.ylabel("Accuracies")
plt.show()

In [ ]:
# Need to do an experiment where we vary the amount of noise value on the test set
frac_noise = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0]

accs = []
losses = []
acc_kps = []
for frac in frac_noise:
    test_acc, test_loss, test_acc_kps = test_func(model_2d, model_lift, test_dataloader, frac_noise =frac)
    accs.append(test_acc)
    losses.append(test_loss)
    acc_kps.append(test_acc_kps)

In [ ]:
# plots PCK for different noise 
fig, ax = plt.subplots()

ax.bar(list(map(str,frac_noise)), accs, width=1, edgecolor="white", linewidth=5)

plt.title("Lifting Model PCK Accuracies with Varying Noises on 2D GT Data")
plt.xlabel("Percentage Noise")
plt.ylabel("Accuracies")
plt.show()

In [7]:

# Code get the first batch of results
with torch.no_grad():
    for data in train_dataloader:
        image = data['file_name']
        X = data['image'].to(device, dtype=torch.float)
        y = data['key_points_3D'].to(device, dtype=torch.float)
        vis = data['visibility']
        
        
        pred_heatmap =  model_2d(X)
        kps2d,_ = get_max_preds(pred_heatmap.detach().cpu().numpy())

        kps2d = kps2d.reshape((len(kps2d), 56))
        pred = model_lift(torch.from_numpy(kps2d).to(device))
        break

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 3-dimensional input of size [3, 256, 256] instead

In [ ]:
sample =80
prediction = pred[sample].cpu().numpy()
actual = (y[sample].cpu()).numpy()
visibility_mask = vis[sample]

mean = sungaya_dataset.means_3d[:,2]
std = sungaya_dataset.std_3d[:,2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

unnormalised_acc = (actual*sungaya_dataset.std_3d)+sungaya_dataset.means_3d
ax, line1 = plot_stick_bug(ax, unnormalised_acc, visibility_mask)

unnormalised_est = unnormalised_acc.copy()
unnormalised_est[:,2] = (prediction*std)+mean
ax, line2 = plot_stick_bug(ax, unnormalised_est, visibility_mask, True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.title(image[sample])
plt.legend(handles=[line1,line2], loc='upper right')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)
# opens external plot
plt.show()